[View in Colaboratory](https://colab.research.google.com/github/YoungestSalon/TIL/blob/master/20180719_Crawling_%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90_v1_0.ipynb)

## 2018.07.19.오전 크롤링#3 과제


**기본과제**

국민청원 첫 페이지 하단의 청원 목록에서 번호, 분류, 제목, 참여인원을 추출하여 DataFrame에 담아주세요.


**선택과제**

국민청원 목록 첫 페이지 주소는 https://www1.president.go.kr/petitions?page=1 입니다. 두번째 페이지 주소는 https://www1.president.go.kr/petitions?page=2 입니다. 처음 10개 페이지에 순차적으로 방문하여 번호, 분류, 제목, 참여인원을 추출하고 그 결과를 DataFrame에 담아주세요.


**선택과제**

2018년 7월 17일 현재 국민청원의 글 번호는 1번부터 238663번까지 부여되어 있습니다. 하지만 중간중간 삭제된 청원이 있어서 실제 청원 수는 238663개보다 적습니다. 309510번 청원글 본문의 URL은 https://www1.president.go.kr/petitions/309510 입니다. URL에서 글번호만 바꾸면 해당 청원의 본문으로 이동할 수 있습니다. 이러한 URL 패턴의 특성을 활용하여 전체 청원 중 임의로 100개 청원의 제목과 본문을 추출하여 DataFrame에 담아주세요.



---



## 기본과제

In [69]:
# 설치를 하고,

!pip install html5lib beautifulsoup4

In [0]:
# HTML 받아오기

from urllib import request

url = "https://www1.president.go.kr/petitions?page=1"
with request.urlopen(url) as f:
  charset = f.headers.get_content_charset()
  html = f.read().decode(charset)

In [0]:
# HTML 문자열을 분석하여 DOM 구성하기

from bs4 import BeautifulSoup

bs = BeautifulSoup(html, 'html5lib')

In [0]:
# 국민청원 1페이지 구조 분석 : 청원 목록을 찾아가는 경로

# body
# div id = "wrap"
# div id = "contents" class = "c_sub.on"
# section id = "cont_view" class = "cs_wrap"
# div class = "cs_area"
# div class = "cs_body"
# div class = "wrap"
# div class = "board.text"
# div class = "b_list.category"
# div class = "bl_body"
# ul
# li
# div class = "bl_wrap"

# 번호 : div class = "bl_no"
# 분류 : div class = "bl_category.cs"
# 제목 : div class = "bl_subject" -> a href = "주소"
# 참여인원 : div class = "bl_agree.cb"

# 청원목록에는 한 페이지 당 15개씩 글이 있음

In [73]:
# 청원 목록의 글 번호만 선택하려면?

# 답변 대기 중인 청원에도 .board.text 클래스가 있고,
# 청원 목록에도 .board.text 클래스가 있다.

# 두 클래스의 차이는, 전자는 wrap 클래스의 직계 자식이 아닌 자손이다.
# 즉, 답변 대기 중인 청원에는 csp_box~ 로 시작하는 클래스가 중간에 하나 더 있다.
# 뒤집어서 생각하면, wrap 클래스의 직계 자식인 board.text 클래스는 단 하나이다.
# 그 단 하나는 청원 목록에 들어 있다.
# 그러므로 우리는 그 wrap 클래스의 직계 자식인 board.text 클래스를 찾으면 된다.
# 그래서 '.wrap'과 '.board.text' 사이에는 '>' 가 하나 더 적혀있다.

# 애란쌤 Comment : 이 방법은 코드는 간결하지만, 원본 HTML의 변경에 취약하다.

bl_no = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_no')

bl_no

[<div class="bl_no"><span class="sound_only">번호 </span>242494</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242493</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242492</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242491</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242490</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242489</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242488</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242487</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242486</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242485</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242484</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242483</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242482</div>,
 <div class="bl_no"><span class="sound_only">번호 </span>242481</div>,
 <div class="bl_no"><span class="s

In [0]:
# bl_no의 앞뒤로 붙어있는 지저분한(?) 텍스트를 지우기 위한 함수
# No(번호) 뿐만 아니라, 분류(category)와 제목(subject)도 비슷한 구조이다.
# 참여인원(agree)은 "명"이 추가로 붙어 있으니 참여인원할 때 추가 작업이 필요.

# '>' 라는 문자를 기준으로 자르고(split), 
# 자른 결과의 4번째('[3]') 요소를 선택해서,
# 다시 '<' 라는 문자 기준으로 자른 다음 1번째('[0]') 요소를 선택하여 출력한다.

def to_return(raw):
  result = raw.split(">")[3].split("<")[0]
  return result

In [76]:
# to_return 함수를 호출해서 bl_no의 앞뒤 텍스트를 지우고 no이라는 리스트에 넣는다.

no = []
i_no = 0   # bl_no가 리스트 타입이기 때문에 1개씩 불러오려면 인덱스가 필요하다.

for num in bl_no:   # bl_no를 1개씩 순차적으로 불러오기 위한 반복문
  num = str(bl_no[i_no])   # bl_no의 요소는 nontype → str으로 바꿔야 to_num 함수에서 split가 가능
  no.append(int(to_return(num)))   # to_return의 결과값은 str이므로 int로 변경해서 저장
  i_no = i_no + 1
  
no

[242494,
 242493,
 242492,
 242491,
 242490,
 242489,
 242488,
 242487,
 242486,
 242485,
 242484,
 242483,
 242482,
 242481,
 242480]

In [77]:
# 청원 목록의 분류만 선택하자. (div class = "bl_category.cs")

bl_category = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_category.cs')
bl_category

[<div class="bl_category cs"><span class="sound_only">분류 </span> 기타</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 인권/성평등</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 기타</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 육아/교육</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 경제민주화</div>,
 <div class="bl_category cs"><span class="sound_only">분류 </span> 정치개혁</div>,

In [78]:
# 분류도 마찬가지로 텍스트를 정리해야 한다.

category = []
i_cate = 0

for catego in bl_category:
  catego = str(bl_category[i_cate])
  category.append(to_return(catego))
  i_cate = i_cate + 1
  
category

[' 기타',
 ' 정치개혁',
 ' 정치개혁',
 ' 정치개혁',
 ' 인권/성평등',
 ' 정치개혁',
 ' 정치개혁',
 ' 기타',
 ' 육아/교육',
 ' 정치개혁',
 ' 정치개혁',
 ' 경제민주화',
 ' 정치개혁',
 ' 육아/교육',
 ' 정치개혁']

In [0]:
# 이번에는 청원 목록의 참여인원(div class = "bl_agree.cb")을 선택한다.

bl_agree = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_agree.cb')
bl_agree

In [80]:
# 참여인원도 텍스트 정리를...
# 텍스트 정리하는 코드 3개는 합칠 수 없으려나 (...)

num_agree = []
i_agree = 0

for agree in bl_agree:
  agree = str(bl_agree[i_agree])
  
  # 참여인원은 글 번호와 다르게 숫자 뒤에 "명"과 빈 칸이 붙어 있어서 지워야 함.
  no_agree = int(to_return(agree).split("명")[0])
  
  num_agree.append(no_agree)
  i_agree = i_agree + 1
  
num_agree

[1, 3, 1, 0, 0, 4, 3, 23, 1, 29, 38, 15, 36, 1, 68]

In [81]:
# 마지막으로 청원 목록의 제목(div class = "bl_subject" -> a href = "주소") 차례.

bl_subject = bs.select('.wrap > .board.text .bl_body ul li .bl_wrap .bl_subject a')
bl_subject

[<a class="cb" href="https://www1.president.go.kr/petitions/314233?navigation=petitions"><span class="sound_only">제목 </span>이재명과 은수미의 조폭 유착에 대한 철저한 수사를 ...</a>,
 <a class="cb" href="https://www1.president.go.kr/petitions/314232?navigation=petitions"><span class="sound_only">제목 </span>성남시  국제 마피아파와 이재명경기도지사 은수미성 ...</a>,
 <a class="cb" href="https://www1.president.go.kr/petitions/314231?navigation=petitions"><span class="sound_only">제목 </span>성남시조폭과 정치인들의 적폐를 청산 해주세요</a>,
 <a class="cb" href="https://www1.president.go.kr/petitions/314230?navigation=petitions"><span class="sound_only">제목 </span>경기도지사 선거 다시하겠습니다</a>,
 <a class="cb" href="https://www1.president.go.kr/petitions/314229?navigation=petitions"><span class="sound_only">제목 </span>파타야살인사건 제대로된 수사청원합니다</a>,
 <a class="cb" href="https://www1.president.go.kr/petitions/314228?navigation=petitions"><span class="sound_only">제목 </span>경기도지사 이재명과 성남시장 은수미 즉각 구속수사 ...</a>,
 <a class="cb" href="https://www1.president.go.kr/petitions/314227?

In [82]:
# 제목도 텍스트 정리가 필요하다.

subject = []
i_subject = 0

for sub in bl_subject:
  sub = str(bl_subject[i_subject])
  subject.append(to_return(sub))
  i_subject = i_subject + 1

subject

['이재명과 은수미의 조폭 유착에 대한 철저한 수사를 ...',
 '성남시  국제 마피아파와 이재명경기도지사 은수미성 ...',
 '성남시조폭과 정치인들의 적폐를 청산 해주세요',
 '경기도지사 선거 다시하겠습니다',
 '파타야살인사건 제대로된 수사청원합니다',
 '경기도지사 이재명과 성남시장 은수미 즉각 구속수사 ...',
 '국회의원 급여를 국민투표를 통해 국민이 결정',
 '이재명에대하여',
 '교육부 장관님꼐 청원합니다.',
 '이재명 경기도지사 해임 후 보궐선거를 희망합니다 ( ...',
 '그것이 알고싶다, 성남시 은수미, 이재명 조폭 연루 ...',
 '성남시  청문회 진행건',
 '문재인 대통령님 철저한 조사  부탁드립니다.',
 '학교 교무실 청소에 관하여',
 '이재명 경기도지사, 성남시, 성남 경찰과 성남 국제마 ...']

In [83]:
# 정리가 끝난 1페이지의 청원목록 데이터를 데이터프레임에 추가한다.

import pandas as pd

df_pg1 = pd.DataFrame(
    {'No': no, 'Category': category, 'Subject': subject, 'Agree': num_agree},
    columns = ['No', 'Category', 'Subject', 'Agree']
)

df_pg1.head(15)

,No,Category,Subject,Agree
0,242494,기타,이재명과 은수미의 조폭 유착에 대한 철저한 수사를 ...,1
1,242493,정치개혁,성남시 국제 마피아파와 이재명경기도지사 은수미성 ...,3
2,242492,정치개혁,성남시조폭과 정치인들의 적폐를 청산 해주세요,1
3,242491,정치개혁,경기도지사 선거 다시하겠습니다,0
4,242490,인권/성평등,파타야살인사건 제대로된 수사청원합니다,0
5,242489,정치개혁,경기도지사 이재명과 성남시장 은수미 즉각 구속수사 ...,4
6,242488,정치개혁,국회의원 급여를 국민투표를 통해 국민이 결정,3
7,242487,기타,이재명에대하여,23
8,242486,육아/교육,교육부 장관님꼐 청원합니다.,1
9,242485,정치개혁,이재명 경기도지사 해임 후 보궐선거를 희망합니다 ( ...,29


### [주의] 정치적인 내용입니다.

7월 22일 일요일 오전 0시 30분 기준 데이터. 동시간대 '그것이 알고싶다'에서 다룬 이재명/은수미 당선자 관련 청원글이 많다.

특히 242480번 글의 Agree 숫자가 놀라워서 일단 한번 박제를... 경기도민, 특히 성남시민이다보니 Agree 한 표 누르고 싶은 심정이다.

![대체 텍스트](https://github.com/YoungestSalon/TIL/blob/master/Crawling_Capture.JPG?raw=true)



## 추가과제 1

전부 다 for문으로 for i in range(0, 10) 처리하면 가능하지 않을까?

* HTML 페이지 번호 바꿔서 받아오는 과정 자동화
* 각 요소의 리스트, 인덱스 번호를 페이지별로 자동 생성하도록 기능 추가
* 각 요소의 불러오기(bs.select문)와 데이터 정제(for문) 자동 반복 기능 추가
* 데이터 프레임 생성해서 데이터 추가하는 과정 자동화

## 추가과제 2

* 페이지 구조 파악 : 제목 / 본문 부분의 CSS Selector는?
* for i in range (0, 100) 처리해서 100번을 반복시킨다.
* URL 주소 랜덤 처리는 import random 해서 random.randint(1, 100) 메소드 사용.